<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/sparsity_not_sorted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
from torchvision import transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
torch.manual_seed(1234)
np.random.seed(1234)

In [6]:
mnist_trainset = datasets.MNIST(root='./data', train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = datasets.MNIST(root='./data', 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

train_dataloader = torch.utils.data.DataLoader(mnist_trainset, 
                                               batch_size=50, 
                                               shuffle=True)

test_dataloader  = torch.utils.data.DataLoader(mnist_testset, 
                                               batch_size=50, 
                                               shuffle=False)

In [7]:
# Define the model 
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear_1 = torch.nn.Linear(784, 256)
        self.linear_2 = torch.nn.Linear(256, 10)
        self.sigmoid  = torch.nn.Sigmoid()

    def forward(self, x):
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid(x)
        pred = self.linear_2(x)

        return pred

In [8]:
def get_activation(model):    
    def hook(module, input, output):
        model.layer_activations = output
    return hook

In [9]:
no_epochs = 100
criterion = torch.nn.CrossEntropyLoss()
def sparsity_trainer(optimizer, model):

    # reset the model 
    train_loss = list()
    test_loss  = list()
    test_acc   = list()
    final_spareness = list()

    # define activation list 

    best_test_loss = 1

    for epoch in range(no_epochs):
        total_train_loss = 0
        total_test_loss = 0

        hidden_layer_activation_list = list()

        # training
        # set up training mode 
        model.train()
        for itr, (image, label) in enumerate(train_dataloader):

            image, label = image.to(device), label.to(device)

            optimizer.zero_grad()

            pred = model(image)

            loss = criterion(pred, label)
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

        total_train_loss = total_train_loss / (itr + 1)
        train_loss.append(total_train_loss)

        # testing 
        # change to evaluation mode 
        model.eval()
        total = 0
        for itr, (image, label) in enumerate(test_dataloader):
            image, label = image.to(device), label.to(device)

            pred = model(image)

            loss = criterion(pred, label)
            total_test_loss += loss.item()

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if label[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

            hidden_layer_activation_list.append(model.layer_activations)

        # this conains activations for all epochs 
        final_spareness.append(hidden_layer_activation_list)

        # caculate accuracy 
        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        # append test loss here 
        total_test_loss = total_test_loss / (itr + 1)
        test_loss.append(total_test_loss)

        print('\nEpoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))

        # if total_test_loss < best_test_loss:
        #     best_test_loss = total_test_loss
            # print("Saving the model state dictionary for Epoch: {} with Test loss: {:.8f}".format(epoch + 1, total_test_loss))
            # torch.save(model.state_dict(), "model.dth")

    sparseness_list = list()

    for single_epoch_spareness in final_spareness:
        single_epoch_spareness = torch.stack(single_epoch_spareness)
        layer_activations_list = torch.reshape(single_epoch_spareness, (10000, 256))

        layer_activations_list = torch.abs(layer_activations_list)  # modified 
        num_neurons = layer_activations_list.shape[1]
        population_sparseness = (np.sqrt(num_neurons) - (torch.sum(layer_activations_list, dim=1) / torch.sqrt(torch.sum(layer_activations_list ** 2, dim=1)))) / (np.sqrt(num_neurons) - 1)
        mean_sparseness_per_epoch = torch.mean(population_sparseness)

        sparseness_list.append(mean_sparseness_per_epoch)

    return test_acc, sparseness_list

# AdaDelta

In [10]:
model_Adadelta = Model()
print("model_Adadelta:", model_Adadelta)
model_Adadelta.to(device)
model_Adadelta.sigmoid.register_forward_hook(get_activation(model_Adadelta))
optimizer_Adadelta = torch.optim.Adadelta(model_Adadelta.parameters(), lr=1.0)
Adadelta_test_acc, Adadelta_sparseness_list = sparsity_trainer(optimizer=optimizer_Adadelta, model=model_Adadelta)

f = open("not_sorted_sparsity_Adadelta.txt", "w")
f.write(str(Adadelta_test_acc)+'\n'+str(Adadelta_sparseness_list)+'\n\n')

f.close()

!cp not_sorted_sparsity_Adadelta.txt /content/drive/MyDrive

model_Adadelta: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/100, Train Loss: 0.43482734, Test Loss: 0.25056328, Test Accuracy: 0.92680000

Epoch: 2/100, Train Loss: 0.21552304, Test Loss: 0.17478676, Test Accuracy: 0.94670000

Epoch: 3/100, Train Loss: 0.15641612, Test Loss: 0.13820602, Test Accuracy: 0.95740000

Epoch: 4/100, Train Loss: 0.12157690, Test Loss: 0.11924306, Test Accuracy: 0.96300000

Epoch: 5/100, Train Loss: 0.09919056, Test Loss: 0.09831684, Test Accuracy: 0.96860000

Epoch: 6/100, Train Loss: 0.08336800, Test Loss: 0.09016022, Test Accuracy: 0.97190000


KeyboardInterrupt: ignored

# AdaGrad

In [11]:
model_Adagrad = Model()
print("model_Adagrad:", model_Adagrad)
model_Adagrad.to(device)
model_Adagrad.sigmoid.register_forward_hook(get_activation(model_Adagrad))
optimizer_Adagrad = torch.optim.Adagrad(model_Adagrad.parameters(), lr=0.1)
Adagrad_test_acc, Adagrad_sparseness_list = sparsity_trainer(optimizer=optimizer_Adagrad, model=model_Adagrad)

f = open("not_sorted_sparsity_Adagrad.txt", "w")
f.write(str(Adagrad_test_acc)+'\n'+str(Adagrad_sparseness_list)+'\n\n')

f.close()

!cp not_sorted_sparsity_Adagrad.txt /content/drive/MyDrive

model_Adagrad: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/100, Train Loss: 0.23439522, Test Loss: 0.12393352, Test Accuracy: 0.96080000

Epoch: 2/100, Train Loss: 0.09499635, Test Loss: 0.09212695, Test Accuracy: 0.97110000

Epoch: 3/100, Train Loss: 0.06664003, Test Loss: 0.08824838, Test Accuracy: 0.97180000

Epoch: 4/100, Train Loss: 0.05111396, Test Loss: 0.07646110, Test Accuracy: 0.97580000

Epoch: 5/100, Train Loss: 0.04075940, Test Loss: 0.07340739, Test Accuracy: 0.97640000

Epoch: 6/100, Train Loss: 0.03315783, Test Loss: 0.06994413, Test Accuracy: 0.97790000

Epoch: 7/100, Train Loss: 0.02713919, Test Loss: 0.07005949, Test Accuracy: 0.97730000

Epoch: 8/100, Train Loss: 0.02278634, Test Loss: 0.06832454, Test Accuracy: 0.97790000

Epoch: 9/100, Train Loss: 0.01949250, Test Loss: 0.06757483, Test Accuracy: 0.97890000

Epoch: 10/100, Train Loss: 

# SGD

In [12]:
model_SGD = Model()
print("model_SGD:", model_SGD)
model_SGD.to(device)
model_SGD.sigmoid.register_forward_hook(get_activation(model_SGD))
optimizer_SGD = torch.optim.SGD(model_SGD.parameters(), lr=0.1)
SGD_test_acc, SGD_sparseness_list = sparsity_trainer(optimizer=optimizer_SGD, model=model_SGD)

f = open("not_sorted_sparsity_SGD.txt", "w")
f.write(str(SGD_test_acc)+'\n'+str(SGD_sparseness_list)+'\n\n')

f.close()

!cp not_sorted_sparsity_SGD.txt /content/drive/MyDrive

model_SGD: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/100, Train Loss: 0.76356382, Test Loss: 0.37123979, Test Accuracy: 0.89470000

Epoch: 2/100, Train Loss: 0.34639586, Test Loss: 0.30502521, Test Accuracy: 0.91280000

Epoch: 3/100, Train Loss: 0.30632597, Test Loss: 0.28052564, Test Accuracy: 0.91830000

Epoch: 4/100, Train Loss: 0.28360342, Test Loss: 0.26220752, Test Accuracy: 0.92460000

Epoch: 5/100, Train Loss: 0.26567984, Test Loss: 0.25403698, Test Accuracy: 0.92850000

Epoch: 6/100, Train Loss: 0.24908113, Test Loss: 0.23463870, Test Accuracy: 0.93200000

Epoch: 7/100, Train Loss: 0.23349987, Test Loss: 0.22112721, Test Accuracy: 0.93650000

Epoch: 8/100, Train Loss: 0.21825378, Test Loss: 0.20715809, Test Accuracy: 0.94200000

Epoch: 9/100, Train Loss: 0.20464664, Test Loss: 0.19649480, Test Accuracy: 0.94210000

Epoch: 10/100, Train Loss: 0.19

# Adam

In [ ]:
model_Adam = Model()
print("model_Adam:", model_Adam)
model_Adam.to(device)
model_Adam.sigmoid.register_forward_hook(get_activation(model_Adam))
optimizer_Adam = torch.optim.Adam(model_Adam.parameters(), lr=0.001)
Adam_test_acc, Adam_sparseness_list = sparsity_trainer(optimizer=optimizer_Adam, model=model_Adam)

f = open("not_sorted_sparsity_Adam.txt", "w")
f.write(str(Adam_test_acc)+'\n'+str(Adam_sparseness_list)+'\n\n')

f.close()

!cp not_sorted_sparsity_Adam.txt /content/drive/MyDrive

model_Adam: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/100, Train Loss: 0.42707376, Test Loss: 0.23229995, Test Accuracy: 0.92990000

Epoch: 2/100, Train Loss: 0.19683726, Test Loss: 0.16562192, Test Accuracy: 0.95000000

Epoch: 3/100, Train Loss: 0.14206681, Test Loss: 0.13353463, Test Accuracy: 0.96060000

Epoch: 4/100, Train Loss: 0.10818527, Test Loss: 0.10605625, Test Accuracy: 0.96830000

Epoch: 5/100, Train Loss: 0.08564620, Test Loss: 0.09401113, Test Accuracy: 0.97100000

Epoch: 6/100, Train Loss: 0.06876795, Test Loss: 0.08297990, Test Accuracy: 0.97430000

Epoch: 7/100, Train Loss: 0.05615342, Test Loss: 0.08354625, Test Accuracy: 0.97440000

Epoch: 8/100, Train Loss: 0.04597417, Test Loss: 0.07344530, Test Accuracy: 0.97780000

Epoch: 9/100, Train Loss: 0.03769751, Test Loss: 0.06873602, Test Accuracy: 0.97840000

Epoch: 10/100, Train Loss: 0.0